In [302]:
import os 
import sys
from pathlib import Path
from bs4 import BeautifulSoup
import json
import requests
import time
from datetime import datetime
import pandas as pd
import numpy as np 

hoops_dir = Path("/Users/danielfriedman/Desktop/hoops")
data_dir = hoops_dir / "data"
lines_dir = data_dir / "lines"
sys.path.append(hoops_dir.as_posix())

In [283]:
url = "https://sportsbook.draftkings.com/leagues/basketball/88670846" # TODO: does that URL change with the date?


In [299]:
def track_lines(url, sleep=30, max_iter=1000):
    all_lines = pd.DataFrame(columns=['home', 'vis', 'home_score', 'vis_score', 'quarter', 'time', 'home_mline', 'vis_mline'])
    start_time = datetime.now().strftime("%m-%d-%Y_%H:%M:%S")
    
    for i in range(max_iter):
        ## Parse the page
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        
        ## Get all Lines
        lines = soup.find_all("tbody", class_="sportsbook-table__body")[0].find_all("tr")

        ## Get the teams
        teams = [line.find("div", class_="event-cell__name-text").text.split(" ")[0] for line in lines]
        
        ## Get the time
        clocks = [line.find("div", class_='event-cell__clock') for line in lines[::2]]
        times = [clock.find_all("span")[0].string if clock else None for clock in clocks]
        quarters = [int(clock.find_all("span")[1].string[0]) if clock else None for clock in clocks] # TODO: OT?

        ## Get the scores
        scores = [line.find("span", class_="event-cell__score") for line in lines]
        scores = [int(score.text) if score else None for score in scores]

        ## Get the lines
        mlines = [int(line.find("span", class_="sportsbook-odds american no-margin default-color").text.replace("+", "")) for line in lines]        

        ## Update lines
        iter_lines = pd.DataFrame(columns=['home', 'vis', 'home_score', 'vis_score', 'quarter', 'time', 'home_mline', 'vis_mline'])
        iter_lines['home'] = teams[1::2]
        iter_lines['vis'] = teams[::2]
        iter_lines['home_score'] = scores[1::2]
        iter_lines['vis_score'] = scores[::2]
        iter_lines['quarter'] = quarters
        iter_lines['time'] = times
        iter_lines['home_mline'] = mlines[1::2]
        iter_lines['vis_mline'] = mlines[::2]        
        all_lines = pd.concat([all_lines, iter_lines]).drop_duplicates()
        all_lines.to_csv(lines_dir / (start_time + ".csv"), index=False)
        print(all_lines)
        print()
        
        ## Sleep
        time.sleep(sleep)
        
        ## Break
        if len(lines) == 0:
            break
    
    ## Return
    return all_lines

lines = track_lines(url, sleep=15, max_iter=1000)


  home  vis home_score vis_score quarter   time home_mline vis_mline
0  POR   GS         32        33       2  11:39        350      -525
1  SAC  DEN         37        40       2   8:21        185      -245

  home  vis home_score vis_score quarter   time home_mline vis_mline
0  POR   GS         32        33       2  11:39        350      -525
1  SAC  DEN         37        40       2   8:21        185      -245
0  POR   GS         32        33       2  11:21        375      -575

  home  vis home_score vis_score quarter   time home_mline vis_mline
0  POR   GS         32        33       2  11:39        350      -525
1  SAC  DEN         37        40       2   8:21        185      -245
0  POR   GS         32        33       2  11:21        375      -575
0  POR   GS         32        33       2  11:17        350      -525

  home  vis home_score vis_score quarter   time home_mline vis_mline
0  POR   GS         32        33       2  11:39        350      -525
1  SAC  DEN         37        4

KeyboardInterrupt: 

In [ ]:
(58+63)/24

In [57]:
url = "https://sportsbook.draftkings.com/event/180233664"
for i in range(1000):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    time_left = soup.find_all(class_="event-cell__time")[0].text
    quarter = soup.find_all(class_="event-cell__period")[0].text    
    print(time_left, "in the", quarter)
    scores = soup.find_all(class_="live-score-body__row--score")
    scores = scores[4].text, scores[9].text
    print(scores)
    mlines = soup.find_all(class_="sportsbook-odds american no-margin default-color")
    print(mlines[0].text, mlines[1].text)
    print()
    time.sleep(15)


5:08 in the 1st Quarter
('18', '10')
-385 +270

5:08 in the 1st Quarter
('18', '11')
-370 +260

5:08 in the 1st Quarter
('18', '12')
-370 +260

5:06 in the 1st Quarter
('18', '12')
-350 +250

4:35 in the 1st Quarter
('18', '12')
-350 +250

4:21 in the 1st Quarter
('18', '12')
-350 +250

4:21 in the 1st Quarter
('18', '12')
-350 +250

4:21 in the 1st Quarter
('19', '12')
-350 +250

4:21 in the 1st Quarter
('19', '12')
-330 +240

4:06 in the 1st Quarter
('19', '12')
-350 +250

4:04 in the 1st Quarter
('19', '12')
-385 +270

3:54 in the 1st Quarter
('19', '12')
-350 +250

3:54 in the 1st Quarter
('19', '12')
-330 +240

3:37 in the 1st Quarter
('19', '12')
-370 +260

3:19 in the 1st Quarter
('19', '14')
-315 +230

3:17 in the 1st Quarter
('19', '14')
-315 +230

3:11 in the 1st Quarter
('19', '14')
-300 +220

2:53 in the 1st Quarter
('19', '17')
-265 +200

2:34 in the 1st Quarter
('22', '17')
-315 +230

2:24 in the 1st Quarter
('22', '17')
-300 +220

2:24 in the 1st Quarter
('22', '17')
-28

KeyboardInterrupt: 

In [58]:
url = "https://sportsbook.draftkings.com/event/180233664"
for i in range(1000):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    time_left = soup.find_all(class_="event-cell__time")[0].text
    quarter = soup.find_all(class_="event-cell__period")[0].text    
    scores = soup.find_all(class_="live-score-body__row--score")
    vis_score, home_score = int(scores[4].text), int(scores[9].text)
    mlines = soup.find_all(class_="sportsbook-odds american no-margin default-color")
    home_line, vis_line = mlines[0].text, mlines[1].text
    mins = 12*(int(quarter[0])-1)
    mins += 11-int(time_left[:3])
    home_lead = home_score - vis_score
    print(f"home_line = {home_line}")
    print(f"vis_line = {vis_line}")
    print(f"home_lead = {home_lead} # {home_score}, {vis_score}")
    print(f"mins = {mins}")
    time.sleep(15)


11:17 in the 2nd Quarter
('33', '30')
-265 +200

11:19 in the 2nd Quarter
('33', '30')
-265 +200

11:10 in the 2nd Quarter
('33', '32')
-265 +200

10:53 in the 2nd Quarter
('36', '32')
-280 +210

10:36 in the 2nd Quarter
('36', '35')
-245 +185

10:21 in the 2nd Quarter
('38', '35')
-265 +200

10:02 in the 2nd Quarter
('38', '35')
-280 +210

10:02 in the 2nd Quarter
('38', '35')
-280 +210

9:45 in the 2nd Quarter
('41', '35')
-350 +250

9:32 in the 2nd Quarter
('41', '35')
-350 +250

9:32 in the 2nd Quarter
('41', '35')
-350 +250

9:32 in the 2nd Quarter
('41', '35')
-330 +240

9:34 in the 2nd Quarter
('41', '35')
-330 +240

9:34 in the 2nd Quarter
('41', '35')
-300 +220

9:24 in the 2nd Quarter
('41', '35')
-350 +250

9:03 in the 2nd Quarter
('41', '35')
-315 +230

8:45 in the 2nd Quarter
('41', '35')
-315 +230

8:39 in the 2nd Quarter
('41', '35')
-300 +220

8:39 in the 2nd Quarter
('41', '36')
-300 +220

8:39 in the 2nd Quarter
('41', '37')
-280 +210

8:33 in the 2nd Quarter
('41', '

KeyboardInterrupt: 

In [64]:
url = "https://sportsbook.draftkings.com/event/180233664"
for i in range(1000):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    time_left = soup.find_all(class_="event-cell__time")[0].text
    quarter = soup.find_all(class_="event-cell__period")[0].text    
    scores = soup.find_all(class_="live-score-body__row--score")
    vis_score, home_score = int(scores[4].text), int(scores[9].text)
    mlines = soup.find_all(class_="sportsbook-odds american no-margin default-color")
    vis_line, home_line = mlines[0].text, mlines[1].text
    mins = 12*(int(quarter[0])-1)
    mins += 11-int(time_left[:1])
    home_lead = home_score - vis_score
    print(f"home_line = {home_line}")
    print(f"vis_line = {vis_line}")
    print(f"home_lead = {home_lead} # ({home_score}, {vis_score})")
    print(f"mins = {mins}")
    time.sleep(15)
    print()


home_line = +260
vis_line = -370
home_lead = -6 # (37, 43)
mins = 16

home_line = +240
vis_line = -330
home_lead = -6 # (37, 43)
mins = 16

home_line = +220
vis_line = -300
home_lead = -4 # (39, 43)
mins = 16

home_line = +210
vis_line = -280
home_lead = -4 # (39, 43)
mins = 17

home_line = +220
vis_line = -300
home_lead = -4 # (39, 43)
mins = 17

home_line = +230
vis_line = -315
home_lead = -4 # (39, 43)
mins = 17

home_line = +230
vis_line = -315
home_lead = -4 # (39, 43)
mins = 17

home_line = +230
vis_line = -315
home_lead = -4 # (39, 43)
mins = 17

home_line = +230
vis_line = -315
home_lead = -4 # (39, 43)
mins = 17

home_line = +230
vis_line = -315
home_lead = -4 # (39, 43)
mins = 17

home_line = +230
vis_line = -315
home_lead = -4 # (39, 43)
mins = 17

home_line = +230
vis_line = -315
home_lead = -4 # (39, 43)
mins = 17

home_line = +230
vis_line = -315
home_lead = -4 # (39, 43)
mins = 17

home_line = +230
vis_line = -315
home_lead = -4 # (39, 43)
mins = 17

home_line = +230
vis

KeyboardInterrupt: 

In [69]:
url = "https://sportsbook.draftkings.com/event/180233806"
for i in range(1000):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    time_left = soup.find_all(class_="event-cell__time")[0].text
    quarter = soup.find_all(class_="event-cell__period")[0].text    
    scores = soup.find_all(class_="live-score-body__row--score")
    vis_score, home_score = int(scores[4].text), int(scores[9].text)
    mlines = soup.find_all(class_="sportsbook-odds american no-margin default-color")
    vis_line, home_line = mlines[0].text, mlines[1].text
    mins = 12*(int(quarter[0])-1)
    mins += 11-int(time_left[:2].replace(":", ""))
    home_lead = home_score - vis_score
    print(f"home_line = {home_line}")
    print(f"vis_lin e = {vis_line}")
    print(f"home_lead = {home_lead} # ({home_score}, {vis_score})")
    print(f"mins = {mins}")
    time.sleep(60)
    print()


home_line = +900
vis_line = -2000
home_lead = -15 # (52, 67)
mins = 23


KeyboardInterrupt: 